<a href="https://colab.research.google.com/github/jdh4/resnet50/blob/master/day5_computer_vision_hackathon_notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction to Machine Learning  
**Computer Vision Hackathon  
Wintersession  
Tuesday, January 24, 2023**

This notebook trains a simple CNN on the MNIST dataset. The code comes from a [PyTorch example on GitHub](https://github.com/pytorch/examples/blob/master/mnist/main.py).

# About Your Colab Session

Learn about the CPU-cores for your session:

In [14]:
cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0xffffffff
cpu MHz		: 2000.170
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmi

In [15]:
import os
num_cores = min(os.cpu_count(), 2)
print(num_cores)

2


Let's see which GPU we are using (probably a Tesla T4):

In [16]:
!nvidia-smi

Tue Jan 24 03:06:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    30W /  70W |   1086MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data Preparation

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

We want to use a GPU when one is available:

In [18]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [19]:
torch.manual_seed(42)
device = torch.device("cuda") if use_cuda else torch.device("cpu")

train_kwargs = {'batch_size': 64}
test_kwargs  = {'batch_size': 1000}
if use_cuda:
    cuda_kwargs = {'num_workers': num_cores, 'pin_memory': True, 'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [27]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.MNIST('/tmp', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('/tmp', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader  = torch.utils.data.DataLoader(dataset2, **test_kwargs)

# Model Definition

Create a Python class called Net that derives from the nn.Module of PyTorch. The \_\_init__() method defines the network layers and regularization method while the forward method describes the forward pass.

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=32,
                               kernel_size=3,
                               stride=1)
        self.conv2 = nn.Conv2d(in_channels=32,
                               out_channels=64,
                               kernel_size=3,
                               stride=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(in_features=9216,
                             out_features=128)
        self.fc2 = nn.Linear(in_features=128,
                             out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

Instantiate the network and move it to the device (which is a GPU when available). Create the optimizer.

In [22]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

In [23]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
           Dropout-3           [-1, 64, 12, 12]               0
            Linear-4                  [-1, 128]       1,179,776
           Dropout-5                  [-1, 128]               0
            Linear-6                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.52
Params size (MB): 4.58
Estimated Total Size (MB): 5.10
----------------------------------------------------------------


# Train and Test Methods

In [24]:
def train(model, device, train_loader, optimizer, epoch):
    model.train() # sets the model in training mode (i.e., dropout enabled)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [25]:
def test(model, device, test_loader):
    model.eval() # sets the model in evaluation mode (i.e., dropout disabled)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train for some number of epochs while reporting the accuracy on the test set periodically:

In [26]:
epochs = 5
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302556
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.421551
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.371526
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.269113
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.106913
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.071669
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.214437
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.072366
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.108041
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.084091

Test set: Average loss: 0.0511, Accuracy: 9824/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.085912
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.027869
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.069379
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.108481
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.045319
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.015593
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.098604
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.012061
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.030015
T